In [155]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from numpy.random import RandomState
import seaborn as sns

# Шаг 1: Загрузка и подготовка данных (выполнено верно)
data0 = pd.read_csv("geo_data_0.csv")
data1 = pd.read_csv("geo_data_1.csv")
data2 = pd.read_csv("geo_data_2.csv")

features_0 = data0.drop(["id", "product"], axis=1)
features_1 = data1.drop(["id", "product"], axis=1)
features_2 = data2.drop(["id", "product"], axis=1)

target_0 = data0["product"]
target_1 = data1["product"]
target_2 = data2["product"]

# Шаг 2.1: Разделение данных (выполнено верно)
train_features_0, valid_features_0, train_target_0, valid_target_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)
train_features_1, valid_features_1, train_target_1, valid_target_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
train_features_2, valid_features_2, train_target_2, valid_target_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

# Шаг 2.2: Обучение и предсказание (создаём отдельные модели!)
model_0 = LinearRegression().fit(train_features_0, train_target_0)
model_1 = LinearRegression().fit(train_features_1, train_target_1)
model_2 = LinearRegression().fit(train_features_2, train_target_2)

predictions_0 = model_0.predict(valid_features_0)
predictions_1 = model_1.predict(valid_features_1)
predictions_2 = model_2.predict(valid_features_2)

# Шаг 2.3: Сохранение результатов (достаточно переменных predictions_X и valid_target_X)
result_0 = pd.DataFrame({"actual": valid_target_0.reset_index(drop=True), "predict": predictions_0})
result_1 = pd.DataFrame({"actual": valid_target_1.reset_index(drop=True), "predict": predictions_1})
result_2 = pd.DataFrame({"actual": valid_target_2.reset_index(drop=True), "predict": predictions_2})

# Шаг 2.4: Расчёт метрик
# Средний запас предсказанного сырья:
mean_pred_0 = predictions_0.mean()
mean_pred_1 = predictions_1.mean()
mean_pred_2 = predictions_2.mean()

# RMSE с корректными скобками:
rmse_0 = mean_squared_error(valid_target_0, predictions_0) ** (1/2)  
rmse_1 = mean_squared_error(valid_target_1, predictions_1) ** (1/2)
rmse_2 = mean_squared_error(valid_target_2, predictions_2) ** (1/2)

# Альтернатива: np.sqrt(mean_squared_error(...))

print(f"Регион 0: Средний запас = {mean_pred_0:.2f}, RMSE = {rmse_0:.2f}")
print(f"Регион 1: Средний запас = {mean_pred_1:.2f}, RMSE = {rmse_1:.2f}")
print(f"Регион 2: Средний запас = {mean_pred_2:.2f}, RMSE = {rmse_2:.2f}")

Регион 0: Средний запас = 92.59, RMSE = 37.58
Регион 1: Средний запас = 68.73, RMSE = 0.89
Регион 2: Средний запас = 94.97, RMSE = 40.03


In [147]:
BUDGET = 1e+07  # 10 млрд рублей бюджет
INCOME_PER_BARREL = 450 # 450 тыс. рублей/тыс. баррелей (так как объём в тыс. баррелей!)
N_EXPLORATION = 500  # Исследуем 500 точек
N_BEST = 200  # Выбираем 200 лучших скважин

min_total_volume = BUDGET / INCOME_PER_BARREL
min_volume_per_well = min_total_volume / N_BEST
print(f"Минимальный средний объём на скважину: {min_volume_per_well:.2f} тыс. баррелей")

def oil_profit_calculation(target, probabilities, count = 200):
    probs_sorted = pd.Series(probabilities).sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return INCOME_PER_BARREL * selected.sum() - BUDGET


print(oil_profit_calculation(result_0["actual"], result_0["predict"]))

Минимальный средний объём на скважину: 111.11 тыс. баррелей
3320826.0431398507


# Bootstrap

In [169]:
import numpy as np
import pandas as pd
from numpy.random import RandomState

# Инициализация состояний для воспроизводимости
state = RandomState(43525)
def bootstrap(targets, predict):
    lost = 0
    value = []
    boost_samples = 1000
    n = len(targets)
    for i in range(boost_samples):
        samples_target = targets.sample(replace = True, random_state = state, n = 500)
        samples_pred = predict[samples_target.index]
        profit_samp = oil_profit_calculation(targets, samples_pred)
        if profit_samp < 0:
            lost += 1
        value.append(profit_samp)
    val = pd.Series(value)
    avg = val.mean()
    upper = val.quantile(0.975)
    lower = val.quantile(0.025)
    risk = lost / 1000
    return val, avg, upper, lower, risk

mean_0,average_0, upper_0, lower_0, risk_0 = bootstrap(result_0['actual'], result_0['predict'])

print("Данные по первому региону")
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_0))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_0, upper_0))
print('Процент риска {:.4%}'.format(risk_0))


Данные по первому региону
Средняя прибыль по региону = 380247.21 тыс.руб.
95% доверительный интервал от -111665.67 до 850629.14 тыс.руб.
Процент риска 6.6000%


In [170]:
mean_1,average_1, upper_1, lower_1, risk_1 = bootstrap(result_1['actual'], result_1['predict'])

print("Данные по первому региону")
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_1))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_1, upper_1))
print('Процент риска {:.4%}'.format(risk_1))

Данные по первому региону
Средняя прибыль по региону = 445838.05 тыс.руб.
95% доверительный интервал от 56717.82 до 841589.53 тыс.руб.
Процент риска 0.9000%


In [171]:
mean_2,average_2, upper_2, lower_2, risk_2 = bootstrap(result_2['actual'], result_2['predict'])

print("Данные по первому региону")
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_2))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_2, upper_2))
print('Процент риска {:.4%}'.format(risk_2))

Данные по первому региону
Средняя прибыль по региону = 399386.93 тыс.руб.
95% доверительный интервал от -120259.57 до 943833.96 тыс.руб.
Процент риска 7.8000%


In [105]:
predictions_0

array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229], shape=(25000,))

In [139]:
result_0

,actual,predict
0,10.038645,95.894952
1,114.551489,77.572583
2,132.603635,77.892640
3,169.072125,90.175134
4,122.325180,70.510088
...,...,...
24995,170.116726,103.037104
24996,93.632175,85.403255
24997,127.352259,61.509833
24998,99.782700,118.180397
